# Sudoku Puzzle Problem

## Import Libraries

In [1]:
import sys
import numpy as np
import pandas as pd

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

## Data Preprocessing

In [2]:
# convert the grapgh into a set
originial_gragh_set ={
    (1,6):1,
    (2,1):2, (2,2):7, (2,5):9, (2,7):5,
    (3,2):8, (3,6):5, (3,9):3,
    (4,3):8, (4,5):3, (4,8):2,
    (5,2):5, (5,4):1, (5,6):2, (5,8):9,
    (6,2):1, (6,5):5, (6,7):7,
    (7,1):5, (7,4):6, (7,8):3,
    (8,3):9, (8,5):1, (8,8):6, (8,9):2,
    (9,4):2
    }
graph_size = 9

## Construct the Model

In [3]:
def sudoku_model(gragh_set, graph_size):
    # Create a model
    model = pyo.ConcreteModel("Sudoku Puzzle")
    
    # Sets of i, j, k
    model.i = pyo.RangeSet(graph_size)
    model.j = pyo.RangeSet(graph_size)
    model.k = pyo.RangeSet(graph_size)
    
    # Decision variables
    model.x = pyo.Var(model.i, model.j, model.k, domain=pyo.Binary)
    
    # Load the graph_set
    for (i, j), k in gragh_set.items():
        model.x[i, j, k].fix(1)

    # Objective function
    model.obj = pyo.Objective(expr=0)
    
    # Constraints
    # row constraint
    @model.Constraint(model.i, model.k)
    def row_constraint(model, i, k):
        return sum(model.x[i, j, k] for j in model.j) == 1
    
    # col constraint
    @model.Constraint(model.j, model.k)
    def col_constraint(model, j, k):
        return sum(model.x[i, j, k] for i in model.i) == 1
    
    # cell constraint
    @model.Constraint(model.i, model.j)
    def cell_constraint(model, i, j):
        return sum(model.x[i, j, k] for k in model.k) == 1
    
    # area constraint
    @model.Constraint(pyo.RangeSet(1,3), pyo.RangeSet(1,3), model.k)
    def area_constraint(model, area_i, area_j, k):
        return sum(model.x[i,j,k] for i in range(3*(area_i-1)+1, 3*area_i+1) for j in range(3*(area_j-1)+1, 3*area_j+1)) == 1
    
    return model
    

## Solve the Model

In [4]:
def solve_model(model):
    SOLVER.solve(model)
#     model.display()

In [5]:
model = sudoku_model(originial_gragh_set, graph_size)
solve_model(model)

## Result Visualization

In [6]:
def visualization(model):
    # initialize
    sudoku_solution = [[0 for j in range(1, 10)] for i in range(1, 10)]
    
    # fill the solution matrix
    for i in model.i:
        for j in model.j:
            for k in model.k:
                if pyo.value(model.x[i, j, k]) == 1:
                    sudoku_solution[i-1][j-1] = k
    
    # print it!
    grid_size = 9
    block_size = 3
    for i in range(grid_size):
        if i % block_size == 0 and i != 0:  
            print('-' * (grid_size * 2 + 3))
        for j in range(grid_size):
            if j % block_size == 0 and j != 0:
                print('|', end=' ')
            print(sudoku_solution[i][j], end=' ')
        print()  


In [7]:
visualization(model)

4 6 5 | 3 7 1 | 2 8 9 
2 7 3 | 8 9 6 | 5 1 4 
9 8 1 | 4 2 5 | 6 7 3 
---------------------
6 9 8 | 7 3 4 | 1 2 5 
7 5 4 | 1 6 2 | 3 9 8 
3 1 2 | 9 5 8 | 7 4 6 
---------------------
5 2 7 | 6 4 9 | 8 3 1 
8 3 9 | 5 1 7 | 4 6 2 
1 4 6 | 2 8 3 | 9 5 7 
